#Análisis de Datos (Películas y Series de Netflix)

##1. ETL (Extracción, transformación y carga)

###1.1 Extracción

In [201]:
from google.colab import drive
import pandas as pd
import chardet

drive.mount('/content/drive', force_remount=True)

ruta_archivo = '/content/drive/MyDrive/Analisis_de_Datos/netflix_titles.csv'
with open(ruta_archivo, 'rb') as f:
    result = chardet.detect(f.read())
try:
  df = pd.read_csv(ruta_archivo, encoding=result['encoding'], sep=",", decimal=".")
  print("Extracción de datos exitosa")
except FileNotFoundError:
  print(f"El archivo no se encuentra en la ruta: {ruta_archivo}")

Mounted at /content/drive
Extracción de datos exitosa


###1.2 Transformación

In [202]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [203]:
# Renombrar columnas en español

df.rename(columns={
    'show_id': 'id',
    'type': 'tipo',
    'title': 'titulo',
    'director': 'directores',
    'cast': 'actores',
    'country': 'pais',
    'date_added': 'fecha_agregado',
    'release_year': 'año_lanzamiento',
    'listed_in': 'genero',
    'description': 'descripcion'
    }, inplace=True)

# Renombrar otros datos en español
df['tipo'] = df['tipo'].str.lower().replace('movie', 'pelicula')
df['tipo'] = df['tipo'].str.lower().replace('tv show', 'serie')


In [204]:
# Conversión de tipos

df['fecha_agregado'] = df['fecha_agregado'].astype('datetime64[ns]')
df['año_lanzamiento'] = df['año_lanzamiento'].astype('Int64')



In [205]:
# Ver qué datos tiene cada columna

print(set(df['tipo']))
print(set(df['rating']))
print(set(df['duration']))


{'serie', 'pelicula'}
{'TV-Y7-FV', 'TV-Y', 'TV-Y7', 'TV-PG', 'G', 'TV-14', 'NR', 'UR', 'TV-G', '84 min', nan, 'TV-MA', '66 min', '74 min', 'NC-17', 'PG-13', 'R', 'PG'}
{'36 min', '62 min', '119 min', '140 min', '104 min', '30 min', '100 min', '58 min', '124 min', '88 min', '74 min', '139 min', '173 min', '8 Seasons', '186 min', '21 min', '42 min', '209 min', '80 min', '103 min', '24 min', '224 min', '3 Seasons', '208 min', '141 min', '149 min', '31 min', '143 min', '157 min', '96 min', '150 min', '112 min', '203 min', nan, '94 min', '22 min', '52 min', '17 Seasons', '106 min', '229 min', '180 min', '1 Season', '159 min', '60 min', '91 min', '182 min', '72 min', '47 min', '63 min', '5 Seasons', '50 min', '109 min', '98 min', '18 min', '156 min', '10 Seasons', '132 min', '81 min', '312 min', '115 min', '77 min', '101 min', '79 min', '38 min', '49 min', '113 min', '273 min', '108 min', '56 min', '153 min', '172 min', '76 min', '125 min', '102 min', '3 min', '160 min', '158 min', '151 min'

In [206]:
# Separar duración en número y tipo de duración (min para películas y temporadas para series)

mask = df['duration'].str.contains('min', na=False)
df.loc[mask, 'duracion'] = df.loc[mask, 'duration'].str.replace(' min', '', regex=False).astype('Int64')
df.loc[mask, 'duracion_tipo'] = 'min'

mask = df['duration'].str.contains('Season', na=False)
df.loc[mask, 'duracion'] = df.loc[mask, 'duration'].str.replace(' Seasons', '').str.replace(' Season', '').astype('Int64')
df.loc[mask, 'duracion_tipo'] = 'Temporadas'

df.drop(columns=['duration'], inplace=True)

print('Transformación de datos exitosa')

Transformación de datos exitosa


In [207]:
# Datos nulos

if (df['directores'].isna().any()):
  print(df['directores'].isna().sum(), 'celdas de la columna directores tienen datos nulos')
if (df['actores'].isna().any()):
  print(df['actores'].isna().sum(), 'celdas de la columna actores tienen datos nulos')
if (df['pais'].isna().any()):
  print(df['pais'].isna().sum(), 'celdas de la columna pais tienen datos nulos')
if (df['fecha_agregado'].isna().any()):
  print(df['fecha_agregado'].isna().sum(), 'celdas de la columna fecha_agregado tienen datos nulos')
if (df['año_lanzamiento'].isna().any()):
  print(df['año_lanzamiento'].isna().sum(), 'celdas de la columna año_lanzamiento tiene datos nulos')
if (df['rating'].isna().any()):
  print(df['rating'].isna().sum(),' celdas de la columna rating tiene datos nulos')
if (df['duracion'].isna().any()):
  print(df['duracion'].isna().sum(), 'celdas de la columna duracion tiene datos nulos')



2634 celdas de la columna directores tienen datos nulos
825 celdas de la columna actores tienen datos nulos
831 celdas de la columna pais tienen datos nulos
10 celdas de la columna fecha_agregado tienen datos nulos
4  celdas de la columna rating tiene datos nulos
3 celdas de la columna duracion tiene datos nulos


In [208]:
# Conversión de datos nulos, no elimino las filas porque los otros datos pueden servir igual

df['directores'] = df['directores'].fillna('Sin dato')
df['actores'] = df['actores'].fillna('Sin dato')
df['pais'] = df['pais'].fillna('Sin dato')
df['fecha_agregado'] = df['fecha_agregado'].fillna(pd.Timestamp("1900-01-01"))
df['rating'] = df['rating'].fillna('Sin dato')
df['duracion'] = df['duracion'].fillna(0)
df['duracion_tipo'] = df['duracion_tipo'].fillna('Sin dato')

print('Transformación de datos nulos exitosa.')

Transformación de datos nulos exitosa.


In [209]:
# Tabla transformada
df.head()

,id,tipo,titulo,directores,actores,pais,fecha_agregado,año_lanzamiento,rating,genero,descripcion,duracion,duracion_tipo
0,s1,pelicula,Dick Johnson Is Dead,Kirsten Johnson,Sin dato,United States,2021-09-25,2020,PG-13,Documentaries,"As her father nears the end of his life, filmm...",90,min
1,s2,serie,Blood & Water,Sin dato,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",2,Temporadas
2,s3,serie,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",Sin dato,2021-09-24,2021,TV-MA,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,1,Temporadas
3,s4,serie,Jailbirds New Orleans,Sin dato,Sin dato,Sin dato,2021-09-24,2021,TV-MA,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",1,Temporadas
4,s5,serie,Kota Factory,Sin dato,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,2021,TV-MA,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,2,Temporadas


###1.3 Carga

In [210]:
# Definir la ruta de salida para guardar el nuevo csv

ruta_salida = '/content/drive/MyDrive/Analisis_de_Datos/netflix_transformado.csv'

# Guardar el dataframe transformado en un nuevo archivo csv en Drive
df.to_csv(ruta_salida, index=False)

print('Carga de datos exitosa')

Carga de datos exitosa


##EDA (Análisis Exploratorio de Datos)

In [211]:
# Estadísticas descriptivas de las columna numéricas
df.describe()

,fecha_agregado,año_lanzamiento,duracion
count,8807,8807.0,8807.0
mean,2019-03-28 17:49:59.114340864,2014.180198,69.823095
min,1900-01-01 00:00:00,1925.0,0.0
25%,2018-04-03 00:00:00,2013.0,2.0
50%,2019-07-01 00:00:00,2017.0,88.0
75%,2020-08-18 00:00:00,2019.0,106.0
max,2021-09-25 00:00:00,2021.0,312.0
std,NaN,8.819312,50.82252


In [212]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               8807 non-null   object        
 1   tipo             8807 non-null   object        
 2   titulo           8807 non-null   object        
 3   directores       8807 non-null   object        
 4   actores          8807 non-null   object        
 5   pais             8807 non-null   object        
 6   fecha_agregado   8807 non-null   datetime64[ns]
 7   año_lanzamiento  8807 non-null   Int64         
 8   rating           8807 non-null   object        
 9   genero           8807 non-null   object        
 10  descripcion      8807 non-null   object        
 11  duracion         8807 non-null   Int64         
 12  duracion_tipo    8807 non-null   object        
dtypes: Int64(2), datetime64[ns](1), object(10)
memory usage: 911.8+ KB


###Preguntas de negocio

In [213]:
# ¿En qué año se realizaron más películas?

peliculas_df = df[df['tipo'] == 'pelicula']

peliculas_por_anio = peliculas_df['año_lanzamiento'].value_counts()
anio_top = peliculas_por_anio.idxmax()
cantidad_top = peliculas_por_anio.max()

print("Año con más películas:", anio_top)
print("Cantidad:", cantidad_top)


Año con más películas: 2017
Cantidad: 767


In [214]:
# ¿En qué año se realizaron más series?
series_df = df[df['tipo'] == 'serie']

series_por_anio = series_df['año_lanzamiento'].value_counts()
año_top = series_por_anio.idxmax()
cantidad_top = series_por_anio.max()

print("Año con más series:", año_top)
print("Cantidad:", cantidad_top)



Año con más series: 2020
Cantidad: 436


In [215]:
# ¿Qué serie y qué película tienen mayor duración? De este análisis vemos que la película con mayor duración en Netflix
# es Black Mirror: Bandersnatch, posiblemente por su modalidad interactiva que permite ir eligiendo la historia.
# En cuanto a la serie de mayor duración, tenemos a Grey's Anatomy, serie que aún sigue en emisión.

pelicula_larga = df[df['tipo'] == 'pelicula'].loc[df['duracion'].idxmax()]
print("Película más larga:")
print(pelicula_larga[['titulo', 'duracion']])

serie_larga = df[df['tipo'] == 'serie']['duracion'].max()
top_series = df[(df['tipo'] == 'serie') & (df['duracion'] == serie_larga)]

print("\nSerie más larga:")
print(top_series[['titulo', 'duracion']])

Película más larga:
titulo      Black Mirror: Bandersnatch
duracion                           312
Name: 4253, dtype: object

Serie más larga:
             titulo  duracion
548  Grey's Anatomy        17


In [216]:
# Cantidad de series y películas por año de lanzamiento. Este análisis nos permite observar que antes de los
# 2000 no se realizaban tantas producciones.

tabla = df.groupby(['año_lanzamiento', 'tipo']).size().unstack(fill_value=0)
print(tabla)

tipo             pelicula  serie
año_lanzamiento                 
1925                    0      1
1942                    2      0
1943                    3      0
1944                    3      0
1945                    3      1
...                   ...    ...
2017                  767    265
2018                  767    380
2019                  633    397
2020                  517    436
2021                  277    315

[74 rows x 2 columns]


In [218]:
# ¿En qué películas y series sus directores también actuaron? De este análisis podemos observar que
# muchos actores dirigen sus propias películas, pero no sucede tanto con las series.

df_validos = df[
    (df['directores'] != 'Sin dato') &
    (df['actores'] != 'Sin dato')
].copy()

df_validos['directores_lista'] = df_validos['directores'].str.split(',')
df_validos['actores_lista'] = df_validos['actores'].str.split(',')

df_validos['directores_lista'] = df_validos['directores_lista'].apply(lambda x: [d.strip() for d in x])
df_validos['actores_lista'] = df_validos['actores_lista'].apply(lambda x: [a.strip() for a in x])

df_validos['directores_actores'] = df_validos.apply(
    lambda row: list(set(row['directores_lista']).intersection(set(row['actores_lista']))),
    axis=1
)

df_resultado = df_validos[df_validos['directores_actores'].map(len) > 0]


resultado = df_resultado[['titulo', 'directores_actores', 'tipo']]

print(resultado.head(100))


                       titulo         directores_actores      tipo
84       Omo Ghetto: the Saga           [Funke Akindele]  pelicula
107          A Champion Heart             [David de Vos]  pelicula
139        Do the Right Thing                [Spike Lee]  pelicula
178             The Interview               [Seth Rogen]  pelicula
188          2 Alone in Paris  [Éric Judor, Ramzy Bedia]  pelicula
...                       ...                        ...       ...
3017                  Jezebel             [Numa Perrier]  pelicula
3077                   Aaviri                [Ravi Babu]  pelicula
3091  El Pepe, a Supreme Life           [Emir Kusturica]  pelicula
3093               The Vendor         [Odunlade Adekola]  pelicula
3112         Being Mrs Elliot              [Omoni Oboli]  pelicula

[100 rows x 3 columns]


In [219]:
# Películas y series realizadas por país y ¿en qué países se realizan más producciones de series y películas?
# De este análisis podemos observar que el país que más producciones tiene, tanto en series como películas es
# Estados Unidos de Norte América, seguido por India en cantidad de películas y por Reino Unido en cuanto a cantidad de series.

paises_validos = df[df['pais'] != 'Sin dato'].copy()

paises_validos['pais_lista'] = paises_validos['pais'].str.split(', ')

paises_separados = paises_validos.explode('pais_lista')

tabla = paises_separados.groupby(['pais_lista', 'tipo']).size().unstack(fill_value=0)

tabla['total'] = tabla.sum(axis=1)

tabla = tabla.sort_values('total', ascending=False)

tabla.index.name = 'pais'

print(tabla.head(5))

tipo            pelicula  serie  total
pais                                  
United States       2751    938   3689
India                962     84   1046
United Kingdom       532    272    804
Canada               319    126    445
France               303     90    393
